In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
from substrate.substrates import Substrate
from matplotlib.ticker import FormatStrFormatter
from tqdm import tqdm


def sorted_glob(pathname):
    files = np.array(glob(pathname))
    return files[np.argsort([int(f.split(".npy")[0].split("_")[-1]) for f in files])]


def plot_frame(phi_files, pol_files, chi, zoom_in=True):
    files = sorted_glob(phi_files)
    phis = [np.load(f) for f in files]
    files = sorted_glob(pol_files)
    pols = [np.load(f) for f in files]

    assert len(phis) == 480

    dpi = 300
    L_box = 50
    pol_min, pol_max = np.nanmin(pols), np.nanmax(pols)

    for k, (phi, p_field_masked) in tqdm(enumerate(zip(phis, pols)), total=len(pols)):
        plt.figure(figsize=(3, 3), dpi=dpi)
        plt.imshow(
            p_field_masked,
            extent=[0, L_box, 0, L_box],
            origin="lower",
            cmap="coolwarm",
            vmin=pol_min,
            vmax=pol_max,
        )
        cbar = plt.colorbar(pad=0.1, format=FormatStrFormatter("%.1f"))
        cbar.set_label("Polarization field " + r"$\mathbb{P}$")
        plt.contour(
            phi,
            levels=[0.5],
            extent=[0, L_box, 0, L_box],
            origin="lower",
            linewidths=[2],
            colors=["black"],
        )
        plt.contour(
            chi,
            levels=[0.5],
            extent=[0, L_box, 0, L_box],
            origin="lower",
            linewidths=[3],
            colors=["black"],
        )

        if zoom_in:
            plt.xlim([15, 35])
            plt.ylim([15, 35])
            plt.axis("off")
        else:
            plt.xlim([0, 50])
            plt.ylim([0, 50])

        plt.savefig(f"{phi_files.split('/phi/*')[0]}/visuals/img_{k}.png")
        plt.close()

In [ ]:
chi = Substrate(N_mesh=200, L_box=50, xi=0.2).two_state_sub(bridge_width=17)
chi = Substrate(N_mesh=200, L_box=50, xi=0.2).rectangular()

id = 26

for r in [0]:
    plot_frame(f"grid_id{id}/run_{r}/phi/*", f"grid_id{id}/run_{r}/pol/*", chi)
    root = f"grid_id{id}/run_{r}"
    cmd = (
        f"ffmpeg -i {root}/visuals/img_%d.png "
        "-b:v 4M -s 600x600 -pix_fmt yuv420p -filter:v 'setpts=PTS' "
        f"{root}/mov_{0}.mp4 -y -hide_banner -loglevel fatal"
    )
    os.system(cmd)